## Phenologist
### ToDO's
Esta dando muito trabalho para rodar fazenda gigante da Bahia. Computador vai explodir. 

1. try it in integer
2. para concatenar resultados, tentar por metrica. 


- tendo problemas com fazendas grandes. Se setar no open_ncs chunks={}, tem que dar um vis[vi] = vis[vi].chunk(dict(time=-1)) no treat_save e ainda meter um .compute() no xr.concat da pheno
    - para a embrapa funcionou, mas ficou mais lento que da forma original (sem chunks={})
    - testar savana
- a fz savana crash o kernel se tentar carregar sem chunks={}


In [1]:
import xarray as xr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
matplotlib.use('TkAgg')
from pathlib import Path
import warnings, os
warnings.filterwarnings("ignore")
from dea_tools.temporal import temporal_statistics, xr_phenology
from dea_tools.spatial import xr_rasterize
from utils import calculate_indices, open_ncs
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import datetime as dt
%matplotlib inline

### define study area

In [2]:
# choose a farm
fazenda = 'fzsavana'

# available areas
farms = {'fzsavana' : {'farmgroupid' : 'a5b9ae81-434a-4224-a81a-3c0ba44102b8', 
                          'file' : '/mnt/geodata/Clientes/0FARMS/FabioRicardi-Barreiras_BA/vetorial/CARs-Fazenda_Savana.gpkg',
                          'parcels' : 'talhoesfarm'},
            'embrapasanca' : {'farmgroupid' : 'fd9239de-65ae-4d0f-a1cf-9c0fd5ac9131', 
                          'file' : '/mnt/geodata/Clientes/Embrapa_SaoCarlos/fazenda_embrapa.gpkg',
                          'parcels' : 'talhoes'},
            'mavi' : {'farmgroupid' : '9a151331-1b50-4804-b416-c217d45d6ac6', 
                          'file' : '/mnt/geodata/Clientes/Mavi/limites_formosa_do_rio_preto.gpkg',
                          'parcels' : 'talhoes'}
                          }

mission = 'Landsat'

#### Functions

In [3]:
# gap-filling, smoothening and save
def treat_save(vis, vi, days, window, folderout, save = True):
    vis[vi] = vis[vi].chunk(dict(time=-1)) #dict(time=-1)
    vis_f1 = vis[vi].interpolate_na( dim = 'time' , 
                                method =  'pchip', # pchip , spline
                                max_gap = np.timedelta64( days , 'D' )
                                ).rolling(time=window).mean(skipna=True)
    
    if save == True:
        Path(f'{folderout}').mkdir( parents = True, exist_ok = True)

        try:
            vis_f1.to_netcdf(f'{folderout}/{vi}_mgap_{days}_w_{window}_v2.nc')
        except:
            os.system(f'rm {folderout}/{vi}_mgap_{days}_w_{window}_v2.nc')
            vis_f1.to_netcdf(f'{folderout}/{vi}_mgap_{days}_w_{window}_v2.nc')
    return vis_f1


# PLOT Phenology - interactive
def plot_phenology( pheno, year, metric, farm, parcels = 'None'):

    da = pheno[metric].sel(time=pheno.time.dt.year == int(year)).squeeze()
    
    if metric in ['vSOS', 'vPOS', 'vEOS', 'AOS', 'Trough', 'LOS']:
        cmap = 'RdYlGn'
    if metric in ['SOS', 'POS', 'EOS']:
        cmap = 'bwr'
    if metric in ['ROG' , 'ROS']:
        cmap = 'magma'
    
    plotao = da.hvplot.quadmesh(x = 'longitude' , y = 'latitude', 
                geo=True, cmap=cmap, robust = True,  height = 600, rasterize = True,
                width = 800) * farm.hvplot(geo=True, line_width = 4, line_color = 'black', color='none') * parcels.hvplot(geo=True,  line_width = 1.5, 
                                                line_color = 'blue',  color='none')
    plotao.opts(opts.Overlay(title=f'{metric} for {year}'))

    return plotao

# PLOT Phenology - interactive
def plot_phenology2( pheno, year, metric, farm, parcels = 'None', width = 12):
    '''
        Learn more
        https://docs.xarray.dev/en/stable/generated/xarray.DataArray.plot.pcolormesh.html
    '''

    da = pheno[metric].sel(time=pheno.time.dt.year == int(year)).squeeze()
    h = np.round(da.shape[0]*width/da.shape[1])
    if h > width:
        orientation = 'vertical'
    else:
        orientation = 'horizontal'

    if metric in ['ROG','ROS']:
        qs = da.quantile([0.01,0.99]).values
        cmap = 'PiYG'
        label = 'LAI / day'
        vmin,vmax = qs[0],qs[1]

    if metric in ['vSOS', 'vPOS', 'vEOS', 'AOS', 'Trough']:
        cmap = 'RdYlGn'
        vmin,vmax = 0, 7.5
        label = 'LAI'

    if metric in ['SOS', 'POS', 'EOS', 'LOS']:
        cmap = 'bwr'
        vmin,vmax = 1, 365
        label = 'day of year'          
        
    fig, ax = plt.subplots(figsize=(width,h))
    da.plot.pcolormesh(x = 'longitude' , y = 'latitude', vmin=vmin, vmax=vmax, ax=ax, cmap=cmap,
                       cbar_kwargs={'orientation':orientation,'shrink':0.6, 'aspect':30,'label': label})
    plt.title(f'{metric} for {year}')
    #plt.yticks(rotation=90)
    farm.plot(ax=ax, edgecolor='black', lw = 2, color='none')
    parcels.plot(ax=ax, edgecolor='black', lw = 1, color='none')


In [4]:
farm = gpd.read_file(farms[fazenda]['file'])
parcels = gpd.read_file(farms[fazenda]['file'], layer = farms[fazenda]['parcels'])
foldernc = f'/mnt/geodata/Clientes/0FARMS/{farms[fazenda]["farmgroupid"]}/'
folderout = f'{foldernc}/results/'

In [5]:
# Load the data
ds = open_ncs(foldernc, mission)
ds1 = ds.resample(time='W').max(skipna=True)
print(ds.dims)
del ds
print(ds1.dims)


FrozenMappingWarningOnValuesAccess({'time': 20, 'latitude': 1201, 'longitude': 1074})
FrozenMappingWarningOnValuesAccess({'time': 21, 'latitude': 1202, 'longitude': 1074})
FrozenMappingWarningOnValuesAccess({'time': 22, 'latitude': 1202, 'longitude': 1074})
FrozenMappingWarningOnValuesAccess({'time': 22, 'latitude': 1201, 'longitude': 1074})
FrozenMappingWarningOnValuesAccess({'time': 20, 'latitude': 1202, 'longitude': 1074})
FrozenMappingWarningOnValuesAccess({'time': 19, 'latitude': 1202, 'longitude': 1074})
FrozenMappingWarningOnValuesAccess({'time': 20, 'latitude': 1201, 'longitude': 1075})
FrozenMappingWarningOnValuesAccess({'time': 22, 'latitude': 1202, 'longitude': 1074})
FrozenMappingWarningOnValuesAccess({'time': 25, 'latitude': 1202, 'longitude': 1074})
FrozenMappingWarningOnValuesAccess({'time': 41, 'latitude': 1202, 'longitude': 1075})
FrozenMappingWarningOnValuesAccess({'time': 45, 'latitude': 1201, 'longitude': 1074})
FrozenMappingWarningOnValuesAccess({'time': 277, 'lati

In [6]:
ds1.chunksizes

Frozen({'time': (3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 7, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 3, 4, 2, 3, 2, 2, 3, 2, 2, 2, 3, 4, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 3, 4, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 5, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 4, 3, 2, 2, 7, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 7, 3, 2, 4, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 5, 2, 2, 5, 2, 5, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 4, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 9, 2, 3, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 2, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 4, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1), 'latitude': (1201,), 'longitude': (1074,)})

In [7]:
vi = 'LAI'
vis = calculate_indices( ds1 , index = [vi], drop = True, collection = mission)
#vis['kNDVI'] = xr.where(vis['kNDVI'] > 0.4, np.nan, vis['kNDVI'])
# vis['NDVI'] = xr.where((vis['NDVI'] > 0.74) | (vis['NDVI'] < -0.05), np.nan, vis['NDVI'])
vis[vi] = xr.where((vis[vi] > 7.4) | (vis[vi] < 0), np.nan, vis[vi])

del ds1


Dropping bands {'nir08', 'blue', 'swir16', 'qa_pixel', 'red'}


### filtering, gap-filling, smoothening,
all that is need to have a nice dataset for **phenology** 

- I've tested smoothening window and 3 and 5 worked well. 

In [8]:
vis_f1 = treat_save(vis = vis, vi = 'LAI', days = 90, window = 7, folderout = folderout, save = True)

### Phenology
The metrics:

            SOS = Date of start of season
            POS = Date of peak of season
            EOS = Date of end of season
            vSOS = Value at start of season
            vPOS = Value at peak of season
            vEOS = Value at end of season
            Trough = Minimum value of season
            LOS = Length of season (Days)
            AOS = Amplitude of season (in value units)
            ROG = Rate of greening
            ROS = Rate of senescence

In [ ]:
# Parcels quick check
datacheck = False
col = None
minmax = (0.5 , 7.5)
di, df = '2023-06','2024-07'

if datacheck:
    for i in range(len(parcels)):
        x,y = parcels.centroid[i].x , parcels.centroid[i].y
        if col == None:
            vis_f1.sel(time=slice(di,df)).sel(longitude = x, latitude = y, method='nearest').plot();
            plt.ylim(minmax)
            plt.title(i);plt.grid();plt.show();plt.close()
    
        else:
            vis_f1.sel(time=slice(di,df)).sel(longitude = x, latitude = y, method='nearest').plot()
            plt.ylim(minmax)
            plt.title(parcels.iloc[i][col]);plt.grid();plt.show();plt.close()
            
            

In [13]:
# the metrics
basic_pheno_stats = [
    "SOS","vSOS",
    "POS", "vPOS",
    "EOS", "vEOS",
    "Trough", "LOS","AOS",
    "ROG","ROS",
]
method_sos = "first"
method_eos = "last"

#### teste safra

In [20]:
# UM OU OUTRO
safrinha = False # FIXME prep for concat
safra = True # SAFRA is prepared for phenology results time-series concatenation

# store results in dict
pheno_results = {}

# get years 
if safrinha:
    suffix = 'safrinha'
    vis_sel = vis_f1.sel(time=vis_f1.time.dt.month.isin([2,3,4,5,6,7,8]))
    years = vis_sel.groupby("time.year") #FIXME acho que a confusão está aqui
    years_int = [y[0] for y in years]

    for year in years_int:
        da = dict(years)[year]
            # calculate stats
        stats = xr_phenology( da, method_sos=method_sos, method_eos=method_eos, stats=basic_pheno_stats,
                            verbose=False, )
        # add results to dict
        pheno_results[str(year)] = stats

if safra:
    suffix = 'safra'
    years_int = np.unique(vis_f1.time.dt.year)
    dates = []
    for year in years_int[:-1]:

        da = vis_f1.sel(time=slice(str(year)+'-06-20',str(year+1)+'-03-01'))
            # calculate stats
        stats = xr_phenology( da, method_sos=method_sos, method_eos=method_eos, stats=basic_pheno_stats,
                            verbose=False )
        # add results to dict
        #datelabel = f'{year}-06-20'
        date = np.datetime64(f'{year}-06-20')
        dates.append(date)
        pheno_results[date] = stats


In [21]:
dates

[numpy.datetime64('2013-06-20'),
 numpy.datetime64('2014-06-20'),
 numpy.datetime64('2015-06-20'),
 numpy.datetime64('2016-06-20'),
 numpy.datetime64('2017-06-20'),
 numpy.datetime64('2018-06-20'),
 numpy.datetime64('2019-06-20'),
 numpy.datetime64('2020-06-20'),
 numpy.datetime64('2021-06-20'),
 numpy.datetime64('2022-06-20'),
 numpy.datetime64('2023-06-20')]

In [22]:
pheno_results[dates[5]]

<xarray.Dataset> Size: 46MB
Dimensions:                       (longitude: 1074, latitude: 1201)
Coordinates:
  * longitude                     (longitude) float64 9kB -45.69 ... -45.39
  * latitude                      (latitude) float64 10kB -11.46 ... -11.79
    card4l:specification          <U2 8B dask.array<chunksize=(), meta=np.ndarray>
    proj:epsg                     int64 8B dask.array<chunksize=(), meta=np.ndarray>
    accuracy:geometric_y_bias     int64 8B dask.array<chunksize=(), meta=np.ndarray>
    landsat:collection_category   <U2 8B dask.array<chunksize=(), meta=np.ndarray>
    full_width_half_max           float64 8B dask.array<chunksize=(), meta=np.ndarray>
    card4l:specification_version  <U3 12B dask.array<chunksize=(), meta=np.ndarray>
    view:off_nadir                int64 8B dask.array<chunksize=(), meta=np.ndarray>
    accuracy:geometric_x_bias     int64 8B dask.array<chunksize=(), meta=np.ndarray>
    epsg                          int64 8B dask.array<chunksize=(), meta=np.ndarray>
    spatial_ref                   int32 4B 4326
Data variables:
    POS                           (latitude, longitude) int16 3MB dask.array<chunksize=(1201, 1074), meta=np.ndarray>
    vEOS                          (latitude, longitude) float32 5MB dask.array<chunksize=(1201, 1074), meta=np.ndarray>
    LOS                           (latitude, longitude) int16 3MB dask.array<chunksize=(1201, 1074), meta=np.ndarray>
    vSOS                          (latitude, longitude) float32 5MB dask.array<chunksize=(1201, 1074), meta=np.ndarray>
    EOS                           (latitude, longitude) int16 3MB dask.array<chunksize=(1201, 1074), meta=np.ndarray>
    vPOS                          (latitude, longitude) float32 5MB dask.array<chunksize=(1201, 1074), meta=np.ndarray>
    Trough                        (latitude, longitude) float32 5MB dask.array<chunksize=(1201, 1074), meta=np.ndarray>
    SOS                           (latitude, longitude) int16 3MB dask.array<chunksize=(1201, 1074), meta=np.ndarray>
    ROG                           (latitude, longitude) float32 5MB dask.array<chunksize=(1201, 1074), meta=np.ndarray>
    AOS                           (latitude, longitude) float32 5MB dask.array<chunksize=(1201, 1074), meta=np.ndarray>
    ROS                           (latitude, longitude) float32 5MB dask.array<chunksize=(1201, 1074), meta=np.ndarray>
Attributes:
    grid_mapping:  spatial_ref

In [23]:
for year in years_int[:-1]:
    year = str(year)
    pheno_results[year].to_netcdf(f'{folderout}/phenology_{year}_{suffix}.nc')

array([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023,
       2024])

In [ ]:
pheno = xr.concat([pheno_results[dates[0]].compute(),pheno_results[dates[1]].compute()], 'time')


In [ ]:
#anos = [str(x) for x in years_int]
pheno = xr.concat([pheno_results[x].compute() for x in dates], 'time')
pheno = pheno.assign_coords({"time" : dates})
pheno.time

In [ ]:
pheno.to_netcdf(f'{folderout}/phenology_{suffix}_v2.nc')

In [ ]:
#pheno['vSOS'].plot(col = 'time', cmap='RdYlGn', vmin = 0.5, vmax = 7, col_wrap = 3, figsize=(16,10))

### plots
#### interactive

In [ ]:
y = 2023
plot_phenology(pheno, y, 'SOS', farm, parcels)

In [ ]:
plot_phenology(pheno, y, 'vSOS', farm, parcels)

In [ ]:
plot_phenology(pheno, y, 'POS', farm, parcels)

In [ ]:
plot_phenology(pheno, y, 'vPOS', farm, parcels)

In [ ]:
plot_phenology(pheno, y, 'EOS', farm, parcels)

In [ ]:
plot_phenology(pheno, y, 'vEOS', farm, parcels)

In [ ]:
plot_phenology(pheno, y, 'LOS', farm, parcels)

In [ ]:
plot_phenology(pheno, y, 'AOS', farm, parcels)

In [ ]:
plot_phenology(pheno, y, 'Trough', farm, parcels)

In [ ]:
plot_phenology(pheno, y, 'ROG', farm, parcels)

In [ ]:
plot_phenology(pheno, y, 'ROS', farm, parcels)

### The matplotlib way

In [ ]:
y = 2022
phenos = ['SOS','vSOS','POS','vPOS','EOS','vEOS','LOS','AOS','Trough','ROG','ROS']
for metric in phenos:
    plot_phenology2(pheno, y, metric, farm, parcels, 8)